## Setup

Installation succeeded 2025-10-02 19:34:18.265847


In [6]:
n = 50  # Number of workers. You're going to want 100 if you need to run mask_and_rechunk
cores = 2  # Number of cpu cores per worker
ram = 15  # GB of ram per worker

warnings.filterwarnings(
    "ignore",
    message="format_bytes is deprecated and will be removed in a future release. Please use dask.utils.format_bytes instead.",
)
warnings.filterwarnings("ignore", message="coroutine 'rpc.close_rpc' was never awaited")
warnings.filterwarnings("ignore", message="Enable tracemalloc to get the object allocation traceback")

gateway = Gateway()
clusters = list(gateway.list_clusters("running"))
for cluster in clusters:
    print(f'''Stopping other cluster: "{cluster.name}"''')
    gateway.stop_cluster(cluster.name)

options = gateway.cluster_options()
options.worker_cores = cores
options.worker_memory = ram
options.image = "Jupyter AGBD"

cluster = gateway.new_cluster(cluster_options=options, shutdown_on_close=False)
print(f'''Started new cluster "{cluster.name}"''')

# scaling options
# manual scaling, the recommended method
cluster.scale(n)

client = cluster.get_client(set_as_default=True)
print(
    f"""Connected to cluster: "{cluster.name}"\nDask Dashboard: {'https://jupyter.chloris-geospatial.com' + cluster.dashboard_link}"""
)

# Sync selected chloris_agbd branch to dask workers.
gitlab_token = toml.load(os.path.join(Path.home(), ".gitlab_token"))
sync_source_to_dask_workers(client, gitlab_token, branch)

Started new cluster "dhub.63d6bd36a4ba4536a5fd22112399e68e"
Connected to cluster: "dhub.63d6bd36a4ba4536a5fd22112399e68e"
Dask Dashboard: https://jupyter.chloris-geospatial.com/services/dask-gateway/clusters/dhub.63d6bd36a4ba4536a5fd22112399e68e/status


2025-10-02 22:06:01,945 - distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


In [ ]:
from dask_gateway import Gateway


gateway = Gateway()
cluster = gateway.connect(gateway.list_clusters()[0].name)
client = cluster.get_client()

print(f"https://jupyter.chloris-geospatial.com{client.dashboard_link}")

https://jupyter.chloris-geospatial.com/services/dask-gateway/clusters/dhub.63d6bd36a4ba4536a5fd22112399e68e/status


ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-2518' coro=<Client._gather.<locals>.wait() done, defined at /srv/conda/envs/notebook/lib/python3.11/site-packages/distributed/client.py:2199> exception=AllExit()>
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.11/site-packages/distributed/client.py", line 2208, in wait
    raise AllExit()
distributed.client.AllExit
ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-2483' coro=<Client._gather.<locals>.wait() done, defined at /srv/conda/envs/notebook/lib/python3.11/site-packages/distributed/client.py:2199> exception=AllExit()>
Traceback (most recent call last):
  File "/srv/conda/envs/notebook/lib/python3.11/site-packages/distributed/client.py", line 2208, in wait
    raise AllExit()
distributed.client.AllExit
ERROR:asyncio:Task exception was never retrieved
future: <Task finished name='Task-2506' coro=<Client._gather.<locals>.wait() done, de

In [ ]:
import os
import rioxarray as rxr
import xarray as xr
import numpy as np
import pandas as pd
from chloris_agbd.aws.s3 import get_s3fs_filesystem
from chloris_agbd.backends.zarr import to_zarr
from dask import delayed, compute

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

## Function

In [ ]:
def make_chm_1m_zarr(site_name:str,
		chunk_size:int = 1000,
		overwrite:bool = True,
		join_type:str = 'inner',
        chm_path:str = 's3://chloris-data-us-west-2/projects/NEON/CHM/'):

	s3fs = get_s3fs_filesystem()

	# Get the files in the CHM dir
	flist = s3fs.ls(chm_path,detail = True)
	flist = [f['Key'] for f in flist if f['type'] == 'file']

	# Get the ones for this site 
	site_files = [f"s3://{x}" for x in flist if site_name in x and x.endswith('.tif')]
	site_years = [int(x.split('_')[1]) for x in site_files]

	chm_list = [] 
	for fname, year in zip(site_files, site_years):

		chm_i = rxr.open_rasterio(fname, mask_and_scale=True, chunks = {'x':chunk_size,'y':chunk_size}).squeeze()
		chm_i = chm_i.expand_dims(year = [year])
		chm_list.append(chm_i)

	chm_da = xr.concat(chm_list, dim='year',join = join_type)
	chm_da = chm_da.chunk({'x': chunk_size, 'y': chunk_size, 'year': -1})
	chm_da = chm_da.rename('canopy_height').astype('float32').to_dataset()

	chm_fname = f"{chm_path.rstrip('/')}/zarrs/{site_name}_chm_1m.zarr"

	to_zarr(chm_fname, chm_da, overwrite=overwrite)

## Run it 

In [ ]:
site_names = ['DELA','LENO','TALL','BONA','DEJU','HEAL','SRER','SJER','SOAP',
              'TEAK','CPER','NIWO','RMNP','DSNY','OSBS','JERC','PUUM','KONZ',
              'UKFS','SERC','HARV','UNDE','BART','JORN','DCFS','NOGP','WOOD',
              'GUAN','LAJA','GRSM','ORNL','CLBJ','MOAB','ONAQ','BLAN','MLBS',
              'SCBI','ABBY','WREF','STEI','TREE','YELL']

# Wrap your function call with dask.delayed
tasks = []
for site_name in site_names:
    task = delayed(make_chm_1m_zarr)(site_name)
    tasks.append(task)

# Trigger parallel execution on the cluster
compute(*tasks)

(None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None)

In [27]:
cluster.shutdown()